In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

listings_df = pd.read_csv('./data/with_amenities_and_estimated_income.csv', low_memory = False)
#listings_df.columns.values
missing_values_count = listings_df.isnull().sum()
missing_values_count

listings_df = listings_df[np.isfinite(listings_df['estimated_income_per_month'])]

#numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

listings_df = listings_df._get_numeric_data()

#thresh = len(listings_df) * .95
#listings_df = listings_df.dropna(thresh = thresh, axis = 1, inplace = True)

listings_df = listings_df[listings_df.columns[listings_df.isnull().mean() < 0.00005]]
listings_df = listings_df.dropna(how='any',axis=0) 
#missing_values_count = listings_df.isnull().sum()
#missing_values_count
#len(listings)


In [2]:
from sklearn.model_selection import train_test_split # splitting data

train_features, test_features, train_outcome, test_outcome = train_test_split(
    listings_df.drop(["estimated_income_per_month"], axis=1),
    listings_df['estimated_income_per_month'],
    test_size=0.30, 
    random_state=11
)



In [7]:
train_features.columns.values

array(['Unnamed: 0', 'id', 'scrape_id', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'latitude', 'longitude',
       'accommodates', 'guests_included', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'calculated_host_listings_count', 'reviews_per_month', 'Other',
       'translation missing: en.hosting_amenity_49', 'Outdoor seating',
       'Pets allowed', 'Pool', 'Electric profiling bed', 'EV charger',
       'Beach essentials', 'High chair', 'Warming drawer',
       'Fixed grab bars for toilet', 'Heat lamps', 'Bed linens',
       'Dishwasher', 'Breakfast', 'Hot tub', 'Kitchenette',
       'Long term stays allowed', 'Rain shower', 'Sound system',
       'Well-lit path to entrance', 'Bathtub', 'Handheld shower head',
       'Espresso machine', 'Game console', 'Wide doorway',
       'Accessible-height bed', 'Washer', 'Single level home',
       'Cleaning be

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

model = LinearRegression()
#backward selection
rfe = RFE(model, 50)
rfe = rfe.fit(train_features, train_outcome)
filterparam = rfe.support_

#finding the total columns
totalparams = train_features.columns
iqmodelparams = totalparams[filterparam]
iqmodelparams

Index(['longitude', 'accommodates', 'Outdoor seating', 'Pool', 'EV charger',
       'Warming drawer', 'Fixed grab bars for toilet', 'Heat lamps',
       'Rain shower', 'Sound system', 'Espresso machine', 'Hair dryer',
       'Firm mattress', 'Projector and screen', 'Shared gym', 'Washer / Dryer',
       'Waterfront', 'Mini fridge', 'Air purifier', 'Table corner guards',
       'Beach view', 'Wine cooler', 'Shared hot tub', 'Terrace', 'Doorman',
       'Paid parking off premises', 'DVD player', 'Hammock', 'Gas oven',
       'Shared pool', 'Stand alone steam shower', 'Outdoor parking',
       'Hot water kettle', 'Walk-in shower', 'Ceiling fan', 'Ski-in/Ski-out',
       'Amazon Echo', 'Roll-in shower', 'Baby monitor', 'Heated towel rack',
       'Mountain view', 'Formal dining area', 'Private hot tub',
       'Standing valet', 'Day bed', 'Private bathroom', 'Exercise equipment',
       'Central air conditioning', 'Gym', 'Jetted tub'],
      dtype='object')

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import make_pipeline           # for making pipelines
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold

# define a pipeline
pipe = make_pipeline(VarianceThreshold(.1), MinMaxScaler(),SelectPercentile(), KNeighborsClassifier())
folds = KFold(n_splits = 10, shuffle = True)

# defines a grid to search through
param_grid = {
              'selectpercentile__percentile':range(10, 30, 5),
              'kneighborsclassifier__n_neighbors':range(1, 10),
              'kneighborsclassifier__weights':["uniform", "distance"]
             }

# performs a grid search of pipeline
knngrid = GridSearchCV(pipe, param_grid,cv = folds, scoring="neg_mean_absolute_error")
knnwholemodel = knngrid.fit(train_features, train_outcome)
knnwholemodel.score(test_features, test_outcome)


-911.9062229904927

In [ ]:
accuracy_score(knnwholemodel.predict(test_features), test_outcome)

AttributeError: 'GridSearchCV' object has no attribute 'named_steps'

In [11]:
from sklearn.ensemble import RandomForestClassifier

pipe = make_pipeline(PolynomialFeatures(), VarianceThreshold(.1), MinMaxScaler(), RandomForestClassifier())
folds = KFold(n_splits = 10, shuffle = True)

param_grid = {
              'polynomialfeatures__degree':range(1, 3),
              'randomforestclassifier__max_depth': range(1,4),
              'randomforestclassifier__min_samples_split': range(2,5),
             }

rfcgrid = GridSearchCV(pipe, param_grid, cv = folds, scoring="neg_mean_absolute_error")
rfcwholemodel = rfcgrid.fit(train_features, train_outcome)
rfcwholemodel.score(test_features, test_outcome)




-1204.1789109766637

In [14]:
accuracy_score(rfcwholemodel.predict(test_features), test_outcome)

0.04278305963699222

In [ ]:
rfcwholemodel.best_params